In [1]:
import pandas as pd
import numpy as np
import os
import time
from IPython.display import clear_output

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
root = 'gdrive/MyDrive'
src_csv = os.path.join(root, 'database_embedding.csv')
dst_csv = os.path.join(root, 'embedding_db.csv')
print('File exists: {}'.format(os.path.exists(src_csv)))

File exists: True


## Transpose data

In [4]:
# df = pd.read_csv(src_csv)
# print('Embedding loaded')

Embedding loaded


In [5]:
# df2 = df.T
# df2.to_csv(dst_csv)

In [6]:
# from google.colab import files
# files.download(dst_csv)

## Calculate similarity

In [4]:
df2 = pd.read_csv(dst_csv, index_col=0)

In [5]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
1014231988_231812.jpg,0.979932,1.165446,-0.473118,-1.260066,1.281455,1.235292,0.416252,-1.730684,0.225623,-0.415296,0.222954,0.295482,-0.027789,0.251379,1.672628,0.348411,-0.255617,-0.861133,-0.725610,-3.201922,-0.466352,-0.326673,0.576172,-2.559891,0.943362,-0.801099,0.211810,1.403465,-1.588117,-0.076943,-0.348962,-0.329823,0.876094,-0.500048,2.263054,0.453805,-1.346471,-2.482449,0.586440,-0.663060,...,1.138011,-0.991440,-0.363166,0.545619,-0.003200,-0.364095,0.963971,0.800756,-2.791887,1.141377,-1.942774,0.181592,1.385560,0.628229,-0.053298,0.081648,1.106875,0.510443,-0.866679,-0.828897,-1.107990,-0.387295,-0.405246,1.619801,-1.136268,0.471542,0.917351,0.824146,1.372707,0.398172,-0.662208,1.302117,0.132062,-0.226168,4.229750,0.078171,0.353426,0.178355,-0.249966,1.245588
1016001172_437323.jpg,0.317535,0.756705,0.448584,-0.463596,-0.664316,0.000986,0.072637,-1.550876,-0.694913,1.189381,-0.118342,1.020214,1.025950,-0.775341,0.253129,1.266778,1.326845,-1.151513,0.912143,-0.854579,0.451873,-0.630974,-0.121439,0.740847,0.151432,0.057868,0.779242,0.276886,1.013950,-0.049069,-0.914032,-1.791093,0.384910,0.556188,1.553044,0.458595,-2.208437,1.107461,0.724636,0.895729,...,-1.562936,-0.015130,-0.750224,1.616508,2.828626,0.420070,0.569133,0.637426,-0.815778,1.706828,-1.961739,-0.967295,-0.363626,-0.118334,0.805446,0.183799,-1.044727,0.461509,0.206586,0.081700,-0.812450,-0.451962,0.058809,2.385701,-0.461046,-0.736411,-0.749516,1.189761,-0.002052,0.587183,-0.465162,1.695829,0.569423,-0.617782,0.611059,-1.409929,1.304691,2.182190,1.435756,0.112308
1233691482_173195.jpg,0.534990,1.190256,-0.132376,0.132695,-2.168387,-0.814362,1.084031,-2.642098,-1.689783,1.796565,-0.403291,1.389410,-0.020837,-0.984456,2.866386,2.298013,1.506042,0.272247,-0.546397,-1.380728,0.199349,-2.159301,1.824462,0.273436,0.632543,-0.885396,-0.645484,1.234602,1.397931,0.368253,-0.917471,-2.093788,-0.982074,0.912606,1.757526,-1.058396,-0.183539,0.480559,-0.065892,0.953125,...,0.128582,0.956938,-0.241993,-0.165745,3.045442,-0.259078,-0.506646,0.967999,-0.589683,1.751752,-0.523548,-0.282933,0.370666,0.431696,0.066272,0.391507,2.104048,-1.268388,-1.209468,0.307181,0.305165,-0.224304,0.129721,2.455866,0.421099,-1.651534,-0.678363,0.118358,0.162727,1.165940,0.001601,1.429232,1.298080,-0.644005,1.261172,-1.201658,-0.550647,2.059888,0.134634,-1.056122
80242521_228815.jpg,-0.805889,-0.008396,-1.574383,0.220078,1.359017,0.283905,0.375282,1.105475,0.246123,0.601768,1.476555,-2.664830,-0.344839,-2.026221,-0.143798,1.528508,-0.115909,-1.326410,-0.972772,0.339253,1.549665,1.758350,1.328320,-0.476214,-1.462514,-1.245182,1.418345,-0.726401,0.197168,1.988694,-0.877514,0.005963,0.505403,-0.903502,1.604792,-0.181541,-0.458340,1.196446,0.476922,-0.038423,...,-0.542310,-1.899193,0.957916,0.845967,1.886023,2.459459,0.257874,0.657861,-1.378995,1.081676,-1.712534,-0.793037,0.854734,-0.430946,-0.616358,0.182622,-0.344455,0.264273,0.521047,0.534475,-0.005042,1.709347,-0.672323,-0.785298,0.225372,0.083978,-2.158528,-0.137544,-0.552079,-1.113982,-1.495565,-0.446212,1.305392,2.068125,-0.177600,-1.655504,0.347487,-0.944348,-1.652171,0.839394
1014182770_305619.jpg,-0.735095,0.664499,-2.234327,-0.682601,1.611458,-1.040077,0.273068,1.044876,-0.319821,1.355419,1.819434,1.199317,1.125481,-2.604085,1.586967,2.345813,0.014587,0.958656,0.024410,1.512587,0.329584,0.069855,-1.332179,-0.226983,-2.201087,-0.483780,0.626243,0.805774,-3.250224,1.794215,-0.749705,-0.217227,-0.143129,-0.062796,0.579161,-0.886202,-0.161447,-1.310759,0.451647,0.127199,...,-0.271841,-2.343923,-0.633862,-0.491495,0.150978,0.292337,0.465830,-2.059417,-0.857285,-0.764288,-1.481406,1.101309,1.644573,0.726181,-1.024306,-0.684264,-0.804228,-0.521736,0.524649,-2.012444,-1.785474,2.34

In [18]:
df2.sort_index(inplace=True)
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
1000046113_190136.jpg,1.392769,0.879834,-0.763906,-0.508705,-1.033225,0.290156,1.886441,-1.048308,-1.392442,1.479466,-1.004000,-0.138799,1.605880,-0.322486,2.814914,1.056868,2.146497,-1.173675,0.045576,-2.758410,-1.176209,-0.834520,1.179324,0.904049,0.699909,0.271152,0.992658,0.722597,0.432572,0.539322,-1.497047,-1.973592,-0.553093,0.955844,2.238655,-0.427605,-1.078934,1.595008,-0.434318,0.795084,...,-0.327594,-1.186719,0.472552,-0.072084,2.353257,0.076660,0.335393,0.927212,-0.189334,1.350123,-0.523627,0.758025,1.162540,0.765741,-0.125558,1.574619,-0.571463,0.281285,-1.942480,-0.787530,0.174175,0.104903,-1.412940,1.567889,-0.180864,-0.316943,-0.706583,-0.410008,-0.289724,0.064437,0.597451,0.190999,1.907559,0.922785,1.444845,-1.740782,-1.088284,1.870509,0.991341,-1.333938
1000046113_190137.jpg,1.826192,0.653667,-0.478559,-0.217734,-0.800203,0.862730,1.872200,-0.809115,-1.083400,1.541001,-1.070368,-0.542637,1.357686,0.103008,2.455723,0.312040,2.164006,-1.257997,0.241710,-2.424635,-1.313937,-0.785017,1.781910,1.551417,1.021294,0.624281,1.519387,0.165303,0.722519,0.269910,-1.576362,-1.771219,-0.548035,0.943056,2.118378,-0.258318,-1.221491,1.525976,-0.440545,1.134334,...,-0.037330,-1.280495,0.745998,0.345555,1.794286,0.219435,0.570403,0.515876,0.036235,0.880858,-0.874348,0.593667,1.292402,1.104643,0.279849,1.875250,-0.811825,0.205266,-2.072738,-1.197880,0.013197,-0.380835,-1.043346,1.439816,-0.175877,0.005688,-0.422738,-0.281662,-0.588080,0.053628,1.122302,0.055444,1.351870,0.574312,1.364401,-1.884376,-1.386176,1.778188,1.425368,-0.762503
1000046113_190138.jpg,1.768410,0.490102,-0.342407,-0.312462,-0.460354,0.890619,1.795373,-0.549559,-1.011468,1.288292,-0.939490,-0.358389,1.637060,-0.261826,2.233266,0.319960,2.383468,-1.369767,0.334636,-2.423119,-1.536256,-0.692821,1.519352,1.612196,0.867990,0.350342,1.426856,0.352924,0.594616,0.405071,-1.371898,-2.004981,-0.656382,0.890259,1.899498,-0.236650,-0.964555,1.706172,-0.427416,1.313294,...,-0.156856,-1.145514,0.681010,0.083697,1.470115,-0.037007,0.717176,0.318482,-0.036396,0.493149,-0.675413,0.880753,1.505588,1.465092,0.392377,1.775715,-1.082360,0.668281,-1.939956,-1.468264,-0.386947,-0.558261,-0.875718,1.598001,-0.117552,-0.250577,-0.427814,-0.276972,-0.826836,0.009521,0.852353,-0.104129,1.345418,0.458302,1.500188,-1.843293,-1.590533,1.587266,1.413511,-0.662049
1000046113_190139.jpg,1.914628,0.405558,-0.441098,-0.523989,-0.455395,0.995524,1.436189,-0.323269,-0.878922,1.308451,-0.978474,-0.292508,1.411895,-0.455979,2.160707,0.324935,2.448864,-1.413082,0.591531,-2.339141,-1.388019,-0.687890,1.387139,1.749917,0.849753,0.269674,1.228706,0.259309,0.676586,0.511606,-1.283976,-2.198924,-0.609314,0.802815,1.800844,0.042964,-1.151575,1.656050,-0.282822,1.335669,...,-0.021604,-1.190593,0.698609,0.277788,1.349348,0.148720,0.592727,0.047045,-0.071741,0.329966,-0.741067,1.011074,1.861582,1.294237,0.558437,1.649724,-1.417564,0.896543,-2.001648,-1.556938,-0.503337,-0.418370,-0.699229,1.505841,-0.221220,-0.362576,-0.273591,-0.382931,-0.735528,-0.010483,0.844567,0.081813,1.408591,0.603813,1.322429,-1.895420,-1.571432,1.474548,1.458241,-0.842150
1000046113_190140.jpg,1.864757,0.310825,-0.658235,-0.542811,-0.540598,0.635829,1.287658,-0.422320,-1.408446,1.763298,-0.947320,-0.270125,1.142501,-0.369036,2.271965,0.586116,2.314505,-1.287538,0.650666,-2.225486,-1.255485,-0.498239,1.251778,1.849258,0.955396,0.554366,1.247671,0.115147,1.079031,0.390222,-1.308651,-2.118104,-0.671712,0.727423,1.881643,0.250826,-1.382242,1.794269,-0.217752,1.209459,...,0.045468,-1.472983,0.813153,0.433152,1.690779,0.147335,0.687473,-0.312628,0.200790,0.348964,-0.435958,0.895880,1.540710,1.240462,0.723083,1.542696,-1.574188,0.738183,-1.422492,-1.330943,-0.425832,-0.163317,-0

In [19]:
df2.loc['1014231988_231812.jpg']

0      0.979932
1      1.165446
2     -0.473118
3     -1.260066
4      1.281455
         ...   
123    0.078171
124    0.353426
125    0.178355
126   -0.249966
127    1.245588
Name: 1014231988_231812.jpg, Length: 128, dtype: float64

In [20]:
def findCosineSimilarity(source_representation, test_representation):
  a = np.matmul(np.transpose(source_representation), test_representation)
  b = np.sum(np.multiply(source_representation, source_representation))
  c = np.sum(np.multiply(test_representation, test_representation))
  return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [21]:
target_folder = os.path.join(root, 'similarity')
data_dict2 = {}
chunks = 1000
chunk_cnt = 0
for idx, (label_1, embedding_1) in enumerate(df2.iterrows()):
  start_time = time.time()
  sub_dict = {}
  for label_2, embedding_2 in df2.iterrows():
    score = round(findCosineSimilarity(np.array(embedding_1), np.array(embedding_2)), 6)
    sub_dict[label_2] = score
  data_dict2[label_1] = sub_dict
  end_time = time.time()
  print('{}: For {} required time was {}s'.format(idx + 1, label_1, round(end_time - start_time, 2)))
  break
  if (idx + 1) % chunks == 0:
    df_tmp = pd.DataFrame.from_dict(data_dict2)
    clear_output(wait=True)
    chunk_filename = 'similarity_{}.csv'.format(chunk_cnt)
    df_tmp.to_csv(os.path.join(target_folder, chunk_filename))
    print('{} saved'.format(chunk_filename))
    chunk_cnt += 1
    data_dict2 = {}

1: For 1000046113_190136.jpg required time was 44.49s


In [11]:
df_sim = pd.read_csv(os.path.join(target_folder, 'similarity_0.csv'), index_col=0)
df_sim.head()

,1014231988_231812.jpg,1016001172_437323.jpg
1014231988_231812.jpg,-0.000000,0.777659
1016001172_437323.jpg,0.777659,0.000000
1233691482_173195.jpg,0.777299,0.507996
80242521_228815.jpg,0.969086,0.967596
1014182770_305619.jpg,0.782044,1.020093
